In [1]:
# STDP unsupervised SNN learning model
# Non-compressed
# Digit recognition (MNIST)

import numpy as np 
import struct
import matplotlib.pyplot as plt
import csv

from brian2 import *

%matplotlib inline

In [2]:
# input signal parameter
# Parameters
# Simulation time [ms]
simT = 100

# Number of Bin (time step)
binN = 10

# Bin period
binT = simT / binN

# Spike probability modifier (max spike per bin [0 ~ 1])
c = 1

# time step
timeStep = np.linspace(0, simT, binN + 1)

In [2]:
# Read MNIST data set (Training)

with open("../data/MNIST/train/train-images.idx3-ubyte", 'rb') as f:
    magic, size = struct.unpack(">II", f.read(8))
    nrows, ncols = struct.unpack(">II", f.read(8))
    data = np.fromfile(f, dtype = np.dtype(np.uint8).newbyteorder('>'))
    data = data.reshape((size, nrows, ncols))
print("Training size = {0}".format(len(data)))

Training size = 60000


In [3]:
# generate randomly distributed spike train using fire rate
# !! relatively fast !!

maxRate = 50    #Hz

imageNo = np.arange(len(data))

pst_all = []

for imageIndex in imageNo:
    pst_img = []
    for rowIndex in range(0, len(data[0])):
        for colIndex in range(0, len(data[0][0])):
            pst_img.append(data[imageIndex][rowIndex][colIndex] * maxRate / 255)
    pst_all.append(pst_img)
    if (imageIndex + 1) % 1000 == 0:
        print("Progress: {0:.2f}%".format(100 * (imageIndex + 1) / len(data)))

pst_rate = np.array(pst_all)

Progress: 1.67%
Progress: 3.33%
Progress: 5.00%
Progress: 6.67%
Progress: 8.33%
Progress: 10.00%
Progress: 11.67%
Progress: 13.33%
Progress: 15.00%
Progress: 16.67%
Progress: 18.33%
Progress: 20.00%
Progress: 21.67%
Progress: 23.33%
Progress: 25.00%
Progress: 26.67%
Progress: 28.33%
Progress: 30.00%
Progress: 31.67%
Progress: 33.33%
Progress: 35.00%
Progress: 36.67%
Progress: 38.33%
Progress: 40.00%
Progress: 41.67%
Progress: 43.33%
Progress: 45.00%
Progress: 46.67%
Progress: 48.33%
Progress: 50.00%
Progress: 51.67%
Progress: 53.33%
Progress: 55.00%
Progress: 56.67%
Progress: 58.33%
Progress: 60.00%
Progress: 61.67%
Progress: 63.33%
Progress: 65.00%
Progress: 66.67%
Progress: 68.33%
Progress: 70.00%
Progress: 71.67%
Progress: 73.33%
Progress: 75.00%
Progress: 76.67%
Progress: 78.33%
Progress: 80.00%
Progress: 81.67%
Progress: 83.33%
Progress: 85.00%
Progress: 86.67%
Progress: 88.33%
Progress: 90.00%
Progress: 91.67%
Progress: 93.33%
Progress: 95.00%
Progress: 96.67%
Progress: 98.33%
Pr

In [ ]:
# Network parameters

# Number of neurons in each layer
# input neurons
n_neuron_input = 784

# excitatory neurons
n_neuron_exc = 100

# inhibitory neurons
n_neuron_inh = 100


# Numaber of synapses after each layer
# input to exc synapses
n_synapse_input = n_neuron_input * n_neuron_exc

# exc to inh synapses
n_synapse_exc = n_neuron_exc

# inh to exc synapses
n_synapse_inh = n_neuron_exc * n_neuron_inh - n_synapse_exc

# 

In [6]:
# Network Construction
start_scope()
num_samples, N_input = pst_array[0].shape

N_class = 100

# 
# Set input timed array (pre-recorded PST) as input layer neurons
input_signal = TimedArray(pst_array[0], dt = 1*ms)
A = 1.5
tau = 2*ms

input_eqs = '''
dv/dt = (A*input_signal(t, i)-v)/tau : 1
'''
input_layer = NeuronGroup(N_input, input_eqs, threshold='v>1', reset='v=0', method='euler')

# Create excitatory layer neurons for classification
excit_eqs = '''
dv/dt = (1-v)/tau
'''

excit_layer = NeuronGroup(N_class, excit_eqs, threshold='v>1', reset='v=0')

# Create inhibitory layer neurons for lateral inhibition
inhib_eqs = '''
dv/dt = (1-v)/tau
'''

inhib_layer = NeuronGroup(N_class, excit_eqs, threshold='v>1', reset='v=0')

# Establish forward one to all STDP synapses between input layer and excitatory layer
input_synapse = Synapses(input_layer, excit_layer, on_pre='v_post+=0.2')
input_synapse.connect()

# Establish forward one to one synapses between excitatory layer and inhibitory layer
excit_forward_synapse = Synapses(excit_layer, inhib_layer, on_pre='v_post+=0.2')
excit_forward_synapse.connect(condition='i==j')

# Establish backward (recurrent) one to other (all except origin) synapses between inhibitory layer and excitatory layer
inhib_backward_synapse = Synapses(inhib_layer, excit_layer, on_pre='v_post-=0.2')
inhib_backward_synapse.connect(condition='i!=j')



EquationError: Parsing failed: 
dv/dt = (1-v)/tau
^
Expected end of text, found 'd'  (at char 1), (line:2, col:1)

dwa
